In [1]:
from model_teams import NeuralNetTeams
from model_pairs import NeuralNetPairs
from itertools import combinations
import random
import pickle

import os
import torch
import numpy as np

In [4]:
import queue

q = None
q = queue.Queue()

def inpu(read_from_queue=False, timeout=10):
    global q
    if read_from_queue:
        return q.get(timeout=timeout)
    else:
        return input()
        
# q.put('1')
# q.put('3')
# q.put('5')
# q.put('0')
while 1:
    try:
        txt = inpu(read_from_queue=(q != None), timeout=3)
        print(txt)
        if txt == 'q':
            break
    except:
        print("Finished printing")
        break



Finished printing


In [2]:
file_name = "pokedex.txt"
print(file_name, '\n')

start = 0
pokedex = {}
pokedex2 = {}

file = open(file_name, 'r')
file2 = open("pokedex_dict.txt", 'w')
    
Lines = file.readlines()

for i in range(len(Lines)):
    index = Lines[start].find('\t')
    num = Lines[start][index + 2:index + 5]
    idx = Lines[start][index + 6:].find('\t')
    poke_name = Lines[start][index + 6: index + 6 + idx].lower()
    
    file2.write(str(start))
#     file2.write(num)
    file2.write('\n')
    file2.write(poke_name)
    file2.write('\n')
    
    pokedex[poke_name] = start
    pokedex2[start] = poke_name
    
    start += 1
    
file.close()
file2.close()

f = open("poke_to_idx.pkl", 'wb')
pickle.dump(pokedex, f)
f.close()

f = open("idx_to_poke.pkl", 'wb')
pickle.dump(pokedex2, f)
f.close()



pokedex

pokedex.txt 



{'bulbasaur': 0,
 'ivysaur': 1,
 'venusaur': 2,
 'charmander': 3,
 'charmeleon': 4,
 'charizard': 5,
 'squirtle': 6,
 'wartortle': 7,
 'blastoise': 8,
 'caterpie': 9,
 'metapod': 10,
 'butterfree': 11,
 'weedle': 12,
 'kakuna': 13,
 'beedrill': 14,
 'pidgey': 15,
 'pidgeotto': 16,
 'pidgeot': 17,
 'rattata': 18,
 'rattata-alola': 19,
 'raticate': 20,
 'raticate-alola': 21,
 'spearow': 22,
 'fearow': 23,
 'ekans': 24,
 'arbok': 25,
 'pikachu': 26,
 'raichu': 27,
 'raichu-alola': 28,
 'sandshrew': 29,
 'sandshrew-alola': 30,
 'sandslash': 31,
 'sandslash-alola': 32,
 'nidoranâ™€': 33,
 'nidorina': 34,
 'nidoqueen': 35,
 'nidoranâ™‚': 36,
 'nidorino': 37,
 'nidoking': 38,
 'clefairy': 39,
 'clefable': 40,
 'vulpix': 41,
 'vulpix-alola': 42,
 'ninetales': 43,
 'ninetales-alola': 44,
 'jigglypuff': 45,
 'wigglytuff': 46,
 'zubat': 47,
 'golbat': 48,
 'oddish': 49,
 'gloom': 50,
 'vileplume': 51,
 'paras': 52,
 'parasect': 53,
 'venonat': 54,
 'venomoth': 55,
 'diglett': 56,
 'diglett-alola'

In [3]:
%%time

dir_name = "Game_Profiles/"
team_name = "trick room"
full_path = dir_name + team_name

print(full_path, '\n\n')
data_teams = np.empty((0,979))
labels_teams = []

data_pairs = np.empty((0,979))
labels_pairs = []

da_stats = {}

start = 0
for filename in os.listdir(full_path):
    file = open(full_path + '/' + filename, 'r')
    print(filename)
    Lines = file.readlines()
    
    for i in range(len(Lines)):
        if(Lines[i] == "--------------------\n"):
            start = i + 1
            break
        else:
            continue
            
    poketeam = []        
    
    # Collect all pokemon on enemy team
    while(start < len(Lines)):
        
        if "No more" in Lines[start]:
            break
        
        # Get lead status
        if "lead" in Lines[start].strip():
            status = 2
        elif "back" in Lines[start].strip():
            status = 1
        else:
            status = 0
        
        start += 1 # Go to name
        
        #Get name
        poke_name = Lines[start].strip().lower()

        start += 13 #skip to next pokemon
        
        print('{:14.14s}'.format(poke_name), '\t', pokedex[poke_name], end='\t')
        if status == 2:
            print("lead")
        elif status == 1:
            print("back")
        else:
            print("absent")
        poketeam.append((pokedex[poke_name], status))
    
    
    # Make input and output vector
    team = torch.zeros(979)
    
    lead = []
    back = []
    absent = []
    
    for (poke, stat) in poketeam:
        team[poke] = 1
        if stat == 2:
            lead.append(poke)
        elif stat == 1:
            back.append(poke)
        else:
            absent.append(poke)
    
    data_teams = np.vstack([data_teams, team])
    labels_teams.append((absent, back, lead) )
    
#     print(poketeam, '\n')

    # Make all combinations
    for combo in combinations(poketeam, 2):  
        pair = torch.zeros(979)
        pair[combo[0][0]] = 1
        pair[combo[1][0]] = 1
        print(combo, end='\t')
        if combo[0][1] == 2 and combo[1][1] == 2:
            print("This is the lead", end='')
        
        isLead = int(combo[0][1] == 2 and combo[1][1] == 2)
        
        data_pairs = np.vstack([data_pairs, pair])
        labels_pairs.append(torch.tensor(isLead))
        print()
    print()
    
    for (poke1, s1) in poketeam:
        
        poke1 = pokedex2[poke1]
        
        if poke1 not in da_stats:
            da_stats[poke1] = {}
        for (poke2, s2) in poketeam:
            poke2 = pokedex2[poke2]
            if poke2 in da_stats[poke1]:
                da_stats[poke1][poke2] += 1
            else:
                da_stats[poke1][poke2] = 1
            
            if s1 == 2 and s2 == 2:
                if "lead" not in da_stats[poke1]:
                    da_stats[poke1]["lead"] = {}
                if poke2 not in da_stats[poke1]["lead"]:
                    da_stats[poke1]["lead"][poke2] = 1
                else:
                    da_stats[poke1]["lead"][poke2] += 1
            
            if s1 == 1 and s2 == 1:
                if "back" not in da_stats[poke1]:
                    da_stats[poke1]["back"] = {}
                if poke2 not in da_stats[poke1]["back"]:
                    da_stats[poke1]["back"][poke2] = 1
                else:
                    da_stats[poke1]["back"][poke2] += 1
            
            if s1 == 0 and s2 == 0:
                if "absent" not in da_stats[poke1]:
                    da_stats[poke1]["absent"] = {}
                if poke2 not in da_stats[poke1]["absent"]:
                    da_stats[poke1]["absent"][poke2] = 1
                else:
                    da_stats[poke1]["absent"][poke2] += 1
                
    

labels_pairs = np.array(labels_pairs)

file.close()

Game_Profiles/trick room 


GameProfile000000
hatterene      	 932	lead
indeedee-f     	 950	absent
liepard        	 562	lead
dusclops       	 391	absent
torkoal        	 356	absent
dhelmise       	 853	absent
((932, 2), (950, 0))	
((932, 2), (562, 2))	This is the lead
((932, 2), (391, 0))	
((932, 2), (356, 0))	
((932, 2), (853, 0))	
((950, 0), (562, 2))	
((950, 0), (391, 0))	
((950, 0), (356, 0))	
((950, 0), (853, 0))	
((562, 2), (391, 0))	
((562, 2), (356, 0))	
((562, 2), (853, 0))	
((391, 0), (356, 0))	
((391, 0), (853, 0))	
((356, 0), (853, 0))	

GameProfile000001
reuniclus      	 638	lead
marowak-alola  	 128	absent
conkeldurr     	 587	back
indeedee-f     	 950	absent
tyranitar      	 278	absent
primarina      	 797	lead
((638, 2), (128, 0))	
((638, 2), (587, 1))	
((638, 2), (950, 0))	
((638, 2), (278, 0))	
((638, 2), (797, 2))	This is the lead
((128, 0), (587, 1))	
((128, 0), (950, 0))	
((128, 0), (278, 0))	
((128, 0), (797, 2))	
((587, 1), (950, 0))	
((587, 1), (278, 0))	
((587

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if (torch.cuda.is_available()):
  print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()
device

GeForce GTX 1660 SUPER


device(type='cuda')

In [5]:
test_size = 0

copy_data = np.copy(data_teams)
copy_labels = labels_teams.copy()

index = np.random.choice(len(copy_data), size=test_size, replace=False).tolist()
index.sort()
index = index[::-1]

test_data = copy_data[index]
test_labels = [copy_labels[x] for x in index]

train_data = np.delete(copy_data, index, axis=0)
train_labels = labels_teams.copy()
for x in index:
    train_labels.pop(x)

print(data_teams.shape)
print(test_data.shape)
print(train_data.shape)

test_data = torch.from_numpy(test_data).to(device)
# test_labels = torch.from_numpy(test_labels).to(device).view(-1, 1)

train_data = torch.from_numpy(train_data).to(device)
# train_labels = torch.from_numpy(train_labels).to(device).view(-1, 1)

(440, 979)
(0, 979)
(440, 979)


In [6]:
def fitTeams(net=None, num_epoch=10, batch_size=10):
    """ Fit a neural net.  Use the full batch size.
    
    @param batch_size: The size of each batch to train on.
    @param num_epoch: The number of epochs to train on
    @param net: A NeuralNet object
    
    @return losses: losses for each batch
    @return net: A NeuralNet object
    """
    
    # Defining the neural network

    if (net == None):
        net = NeuralNetTeams(0.01)
    net.to(device)
    
    # Training

    losses = []
    for epoch in range(num_epoch):
        # Training
        i = 0
        print("epoch:", epoch)
        while (i + 1) * batch_size < len(train_data) + 1:
        
            batch = train_data[i * batch_size: (i + 1) * batch_size]
            target = train_labels[i * batch_size: (i + 1) * batch_size]
        
            batch = batch.float().to(device)
                    
            loss = net.step(batch, target)
            losses.append(loss)
            print('Loss', i, ':', "{:.5f}".format(loss), end='\r')

            i += 1
        print()
        
    return losses, net

In [7]:
%%time
losses, mynet = fitTeams(num_epoch=100)
mynet.save_model('sample_model')

epoch: 0
Loss 43 : 39.13279
epoch: 1
Loss 43 : 38.89562
epoch: 2
Loss 43 : 38.70799
epoch: 3
Loss 43 : 38.52142
epoch: 4
Loss 43 : 38.28513
epoch: 5
Loss 43 : 37.94759
epoch: 6
Loss 43 : 37.54491
epoch: 7
Loss 43 : 37.02246
epoch: 8
Loss 43 : 36.44516
epoch: 9
Loss 43 : 35.85178
epoch: 10
Loss 43 : 35.24728
epoch: 11
Loss 43 : 34.60780
epoch: 12
Loss 43 : 34.01104
epoch: 13
Loss 43 : 33.41122
epoch: 14
Loss 43 : 32.81676
epoch: 15
Loss 43 : 32.23680
epoch: 16
Loss 43 : 31.65445
epoch: 17
Loss 43 : 31.06144
epoch: 18
Loss 43 : 30.43991
epoch: 19
Loss 43 : 29.82010
epoch: 20
Loss 43 : 29.16968
epoch: 21
Loss 43 : 28.49840
epoch: 22
Loss 43 : 27.84770
epoch: 23
Loss 43 : 27.17754
epoch: 24
Loss 43 : 26.48306
epoch: 25
Loss 43 : 25.80577
epoch: 26
Loss 43 : 25.11288
epoch: 27
Loss 43 : 24.47896
epoch: 28
Loss 43 : 23.79476
epoch: 29
Loss 43 : 23.17146
epoch: 30
Loss 43 : 22.47160
epoch: 31
Loss 43 : 21.85421
epoch: 32
Loss 43 : 21.23751
epoch: 33
Loss 43 : 20.63861
epoch: 34
Loss 43 : 20.0

In [8]:
x = random.randint(0, len(data_teams))
print(x)
output = mynet(torch.from_numpy(data_teams[x]).float().to(device))

print(output[0, labels_teams[x][0]])
print(output[0, labels_teams[x][1]])
print(output[0, labels_teams[x][2]])

print(labels_teams[x])
for i in labels_teams[x]:
    for j in i:
        print('{:8.12s}'.format(pokedex2[j]), '\t', j)

370
tensor([[9.5449e-01, 4.7986e-04, 4.5030e-02],
        [8.9770e-01, 9.8909e-02, 3.3879e-03]], device='cuda:0',
       grad_fn=<IndexBackward>)
tensor([[1.3222e-01, 8.6728e-01, 5.0752e-04],
        [3.2738e-03, 9.0591e-01, 9.0811e-02]], device='cuda:0',
       grad_fn=<IndexBackward>)
tensor([[0.0277, 0.0372, 0.9352],
        [0.0154, 0.0083, 0.9764]], device='cuda:0', grad_fn=<IndexBackward>)
([600, 794], [278, 884], [887, 959])
whimsicott 	 600
incineroar 	 794
tyranitar 	 278
rillaboom 	 884
cinderace 	 887
duraludon 	 959


In [9]:
team = torch.zeros(979)
indices = [957, 583, 857, 5, 391, 659]
for i in indices:
    team[i] = 1

output = mynet(team.float().to(device))

print("pokemon\t\tabsent\treserve\tlead")
print('--------------------------------------')

for i in indices:
    print('{:4.12s}'.format(pokedex2[i]), end='\t')
    for o in output[0, i].tolist():
        print('{:.4f}'.format(o), end='\t')
    print()


lead_scores = []


for i in indices:
    lead_scores.append((i, output[0, i] ))
    
lead_scores.sort(key=lambda x:-x[1][2])
print()
print(pokedex2[lead_scores[0][0]], pokedex2[lead_scores[1][0]])

pokemon		absent	reserve	lead
--------------------------------------
dracovish	0.4647	0.1581	0.3772	
excadrill	0.9876	0.0037	0.0087	
tapu koko	0.2715	0.1888	0.5397	
charizard	0.3720	0.2878	0.3402	
dusclops	0.7833	0.1481	0.0687	
ferrothorn	0.4068	0.5727	0.0206	

tapu koko dracovish


In [10]:
test_size = 0

copy_data = np.copy(data_pairs)
copy_labels = np.copy(labels_pairs)

index = np.random.choice(len(copy_data), size=test_size, replace=False).tolist()

test_data = copy_data[index]
test_labels = copy_labels[index]

train_data = np.delete(copy_data, index, axis=0)
train_labels = np.delete(copy_labels, index, axis=0)

print(data_pairs.shape)
print(test_data.shape)
print(train_data.shape)
print(train_labels.shape)

test_data = torch.from_numpy(test_data).to(device)
test_labels = torch.from_numpy(test_labels).to(device).view(-1, 1)

train_data = torch.from_numpy(train_data).to(device)
train_labels = torch.from_numpy(train_labels).to(device).view(-1, 1)

(6582, 979)
(0, 979)
(6582, 979)
(6582,)


In [17]:
def fitPairs(net=None, num_epoch=1000, batch_size=10):
    """ Fit a neural net.  Use the full batch size.
    
    @param batch_size: The size of each batch to train on.
    @param num_epoch: The number of epochs to train on
    @param net: A NeuralNet object
    
    @return losses: losses for each batch
    @return net: A NeuralNet object
    """
    
    # Defining the neural network

    if (net == None):
        net = NeuralNetPairs(0.0005)
    net.to(device)
    
    # Training

    losses = []
    for epoch in range(num_epoch):
        # Training
        i = 0
        print("epoch:", epoch)
        while (i + 1) * batch_size < len(train_data) + 1:
        
            batch = train_data[i * batch_size: (i + 1) * batch_size]
            target = train_labels[i * batch_size: (i + 1) * batch_size]
        
            batch = batch.float().to(device)
            target = target.float().to(device)
                    
            loss = net.step(batch, target)
            losses.append(loss)
            if i % 25 == 0:
                print('Loss', i, ':', "{:.5f}".format(loss), end='\r')

            i += 1
        print()
        
    return losses, net

In [87]:
%%time
losses, mynet2 = fitPairs(mynet2)

epoch: 0
Loss 650 : 0.06789
epoch: 1
Loss 650 : 0.06791300 : 0.98737
epoch: 2
Loss 650 : 0.06785
epoch: 3
Loss 650 : 0.06769
epoch: 4
Loss 650 : 0.06808.57879575 : 0.05229
epoch: 5
Loss 650 : 0.06778
epoch: 6
Loss 650 : 0.06788
epoch: 7
Loss 650 : 0.06782.46958
epoch: 8
Loss 650 : 0.06803600 : 0.83179
epoch: 9
Loss 650 : 0.06811
epoch: 10
Loss 650 : 0.06808
epoch: 11
Loss 650 : 0.06833
epoch: 12
Loss 650 : 0.06823
epoch: 13
Loss 650 : 0.06828
epoch: 14
Loss 650 : 0.06830 : 0.39436450 : 0.19208
epoch: 15
Loss 650 : 0.06830
epoch: 16
Loss 650 : 0.06795
epoch: 17
Loss 650 : 0.06793
epoch: 18
Loss 650 : 0.06799
epoch: 19
Loss 650 : 0.06807
epoch: 20
Loss 650 : 0.068105 : 0.57955
epoch: 21
Loss 650 : 0.06813375 : 0.96126
epoch: 22
Loss 650 : 0.06816
epoch: 23
Loss 650 : 0.068180.05185
epoch: 24
Loss 650 : 0.06802
epoch: 25
Loss 650 : 0.06788
epoch: 26
Loss 650 : 0.06818
epoch: 27
Loss 650 : 0.06801
epoch: 28
Loss 650 : 0.06790
epoch: 29
Loss 650 : 0.06789: 0.72448
epoch: 30
Loss 650 : 0.068

Loss 650 : 0.06742150 : 0.90701600 : 0.83507
epoch: 228
Loss 650 : 0.06764
epoch: 229
Loss 650 : 0.06729
epoch: 230
Loss 650 : 0.06753
epoch: 231
Loss 650 : 0.067620.39201
epoch: 232
Loss 650 : 0.06749
epoch: 233
Loss 650 : 0.06738125 : 0.12841625 : 0.03991
epoch: 234
Loss 650 : 0.06725
epoch: 235
Loss 650 : 0.06736550 : 0.04764
epoch: 236
Loss 650 : 0.06703600 : 0.83461
epoch: 237
Loss 650 : 0.06736150 : 0.90695375 : 0.96543
epoch: 238
Loss 650 : 0.06728 : 0.72185 0.04514
epoch: 239
Loss 650 : 0.06700225 : 0.39226450 : 0.18614
epoch: 240
Loss 650 : 0.06689 : 0.47123 0.02758
epoch: 241
Loss 650 : 0.066830.05193
epoch: 242
Loss 650 : 0.067080.90622
epoch: 243
Loss 650 : 0.06687225 : 0.391820.18589
epoch: 244
Loss 650 : 0.06701 : 0.47056250 : 0.13097
epoch: 245
Loss 650 : 0.066825 : 0.57907: 0.04740
epoch: 246
Loss 650 : 0.066725 : 0.57867
epoch: 247
Loss 650 : 0.06699 0.96580
epoch: 248
Loss 650 : 0.06672425 : 0.04729
epoch: 249
Loss 650 : 0.06699
epoch: 250
Loss 650 : 0.06678 0.02743
e

Loss 650 : 0.06639175 : 0.71413
epoch: 657
Loss 650 : 0.06639475 : 0.39997
epoch: 658
Loss 650 : 0.066360 : 0.69407275 : 0.77687
epoch: 659
Loss 650 : 0.066030.74825
epoch: 660
Loss 650 : 0.06609450 : 0.17388
epoch: 661
Loss 650 : 0.06637 : 0.46657 0.12955475 : 0.39997
epoch: 662
Loss 650 : 0.06631: 0.69351275 : 0.77670
epoch: 663
Loss 650 : 0.0660600 : 0.28112
epoch: 664
Loss 650 : 0.06587
epoch: 665
Loss 650 : 0.06588475 : 0.39986
epoch: 666
Loss 650 : 0.06595
epoch: 667
Loss 650 : 0.06569 : 0.03946
epoch: 668
Loss 650 : 0.06594
epoch: 669
Loss 650 : 0.06590
epoch: 670
Loss 650 : 0.06586
epoch: 671
Loss 650 : 0.06578
epoch: 672
Loss 650 : 0.065870 : 0.69392
epoch: 673
Loss 650 : 0.06584
epoch: 674
Loss 650 : 0.06586425 : 0.04716
epoch: 675
Loss 650 : 0.06574
epoch: 676
Loss 650 : 0.06551
epoch: 677
Loss 650 : 0.06560
epoch: 678
Loss 650 : 0.06600275 : 0.77653
epoch: 679
Loss 650 : 0.0657300 : 0.28006 : 0.83980
epoch: 680
Loss 650 : 0.06574 0.71565400 : 0.04347
epoch: 681
Loss 650 : 0

In [95]:
x = random.randint(0, len(data_pairs) - 15)
print(x, '\n')

output = mynet2(torch.from_numpy(data_pairs[x:x+15]).float().to(device))
answers = labels_pairs[x:x+15] 

print("Predicted\tActual")
for i in range(len(answers)):
    print('{:.5f}\t\t{:d}'.format(output[i].item(), answers[i]))

1007 

Predicted	Actual
0.21886		0
0.08889		0
0.07793		0
0.05050		0
0.02123		0
0.10304		0
0.11830		0
0.05637		0
0.05357		0
0.17975		0
0.05407		0
0.72564		1
0.14438		0
0.42556		1
0.02290		0
